# "Voir aussi" feature implementation

The purpose of this notebook is to find similarity between dataset in order to build recommendation.

More datasets are visited together in a same session, closer they are from each other.

We checked both session & API.

In [1]:
import json
from collections import defaultdict, Counter
import pickle
import glob
import os
import itertools
from IPython.display import clear_output
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

from src import utils

# Parse actions to dataset ids

First, we need to group visited URL by type. For instance, we want to group .fr/.en/.es for the same actions.

By action, we mean "search, posts, actions, goal, ..".

In [2]:
source_dir = 'deduplicated_logs'
target_dir = 'parsed_logs'

filenames = glob.glob(source_dir + '/*')

# Compute monthly filename.json (e.g. : 2017-12-26)
a_month_ago = pd.datetime.today() + pd.DateOffset(months=-1)
date_range = pd.date_range(a_month_ago, pd.datetime.today()).strftime('%Y-%m-%d')
monthly_files = ['deduplicated_logs/' + file + '.json' for file in date_range]

unhandled_actions = []
visits_parsed = []
for filename in monthly_files:
    with open(filename, 'r') as f:
        visits = json.load(f)
    visits_parsed_file, unhandled_actions_file = utils.parse_visits(visits)
    visits_parsed += visits_parsed_file
    unhandled_actions += unhandled_actions_file
    with open(os.path.join(target_dir, os.path.basename(filename)), 'w') as f:
        json.dump(visits_parsed_file, f)
with open(os.path.join(target_dir, os.path.basename('all_visits_parsed_file')), 'w') as f:
    json.dump(visits_parsed, f)

In [3]:
len(visits_parsed)

283243

In [4]:
len(unhandled_actions)

171242

`unhandled_actions` contains mainly "territoires", "avatar" and odd URL. Majority of the datasets were correctly parsed so let's keep going.

`visits_parsed` is a list of list of tuples : `[[(a, b), (a,c)], [(c,d)], [(a,c)], ...]`

each tuple is a (type, name) were `type` can be: `id` (=API community), `keyword` (=keyword search), `slug_or_id` (dataset_API or dataset) and name is the value (e.g. a dataset name).

API Community eg : https://www.data.gouv.fr/api/1/datasets/community_resources/?sort=-created_at&page_size=10&dataset=58e53811c751df03df38f42d&lang=fr&_=1512404531050

-> "58e53811c751df03df38f42d" correspond au RNA, mais quel diff avec api ?


We will focus our study on `slug_or_id`.

In [5]:
# Dict session_num -> list_of_action_tuple
visits_parsed_as_dict = dict(zip(range(0,len(visits_parsed)), visits_parsed))
visits_parsed_as_dict[2]

[('slug_or_id', 'geofla-limites-des-communes-en-france'),
 ('slug_or_id', 'geofla-limites-des-communes-en-france'),
 ('slug_or_id', 'geofla-limites-des-communes-en-france'),
 ('slug_or_id',
  'decoupage-administratif-communal-francais-issu-d-openstreetmap'),
 ('slug_or_id',
  'decoupage-administratif-communal-francais-issu-d-openstreetmap'),
 ('slug_or_id', 'communes-nouvelles-et-fusions-2017'),
 ('slug_or_id', 'communes-nouvelles-et-fusions-2017'),
 ('slug_or_id',
  'decoupage-administratif-communal-francais-issu-d-openstreetmap'),
 ('slug_or_id', 'cantons-3')]

In [6]:
# Dict session_num -> set() of visited datasets (by API or through web)
session_datasets = {}
for key, value in visits_parsed_as_dict.items():
    if len(value) > 0:
        unique_list = set(tup[1] for tup in value if tup[0] == "slug_or_id")
        if len(unique_list) > 1 : # list of at least 2 different datasets
            session_datasets[key] = unique_list
    else:
        pass # We don't care about empty session

In [7]:
len(session_datasets)

29849

### First, let's replace dataset-id by their name (or slug) in order. All of them are supposedly unique so no problem.

To do so, we first need to import the list of all datasets (datasets.csv) il order to get the mapping.

In [8]:
datasets_ref = pd.read_csv('./list_of_all_datasets_dgfr/datasets-2017-12-13-18-23.csv', sep=';')
# Mapping id -> slug
datasets_id_slug = datasets_ref.set_index('id')['slug'].to_dict()

In [9]:
# Replace all dataset_id by dataset_slug (caution: we keep the same dict name)
session_datasets = {key: [datasets_id_slug.get(el, el) for el in value] for key, value in session_datasets.items()}

### Now that we have a dict of set of datasets, let's count how many pairwise datasets appears together.

For example

In [10]:
# Two lists
L1 = ['a', 'b', 'c', 'd'] 
L2 =  ['b', 'a', 'c'] 
# We want : (a, b): 2 (a, c): 2 (b, c): 2, (b, d): 1 (c, d):1

# Compute combination (we need to sort beforehand !)
L1_comb = list(itertools.combinations(np.sort(L1), 2)) # (ab) (ac) (ad) (bc) (bd) (cd)
L2_comb = list(itertools.combinations(np.sort(L2), 2)) # (ba) (bc) (ac)

# Counter tuple
c1 = Counter(L1_comb)
print('L1=  %s' % c1)
c2 = Counter(L2_comb)
print('Somme= %s' % (c1 + c2))

L1=  Counter({('a', 'b'): 1, ('a', 'c'): 1, ('a', 'd'): 1, ('b', 'c'): 1, ('b', 'd'): 1, ('c', 'd'): 1})
Somme= Counter({('a', 'b'): 2, ('a', 'c'): 2, ('b', 'c'): 2, ('a', 'd'): 1, ('b', 'd'): 1, ('c', 'd'): 1})


In [11]:
# Could be optimize. Caution : ~10min.
pairwise_count = Counter()
c = 0
for key, val in session_datasets.items():
    val_as_list = np.sort(list(val))
    val_combinations = list(itertools.combinations(val_as_list, 2))
    pairwise_count += Counter(val_combinations)
    c += 1
    clear_output(wait=True)
    print('%s/%s' % (c, len(session_datasets.items())))

29849/29849


In [12]:
pairwise_count.most_common(5)

[(('cadastre', 'pci-vecteur-plan-cadastral-informatise'), 1682),
 (('base-officielle-des-codes-postaux',
   'correspondance-entre-les-codes-postaux-et-codes-insee-des-communes-francaises'),
  413),
 (('base-sirene-des-entreprises-et-de-leurs-etablissements-siren-siret',
   'base-sirene-des-entreprises-et-de-leurs-etablissements-siren-siret'),
  400),
 (('associations', 'repertoire-national-des-associations-rna'), 368),
 (('base-officielle-des-codes-postaux', 'geofla-r'), 317)]

In [13]:
paired_df = pd.DataFrame(list(pairwise_count.items()), columns=['paired_datasets', 'co_occurence'])
paired_df.sort_values('co_occurence', ascending=False, inplace=True)
paired_df.head()

,paired_datasets,co_occurence
1060,"(cadastre, pci-vecteur-plan-cadastral-informat...",1682
8450,"(base-officielle-des-codes-postaux, correspond...",413
1195,(base-sirene-des-entreprises-et-de-leurs-etabl...,400
2365,"(associations, repertoire-national-des-associa...",368
8633,"(base-officielle-des-codes-postaux, geofla-r)",317


### Now it could be interesting to have : for each datasets, all _twins_ ordered by co-occurence

In [14]:
paired_df[['dataset_1', 'dataset_2']] = paired_df.paired_datasets.apply(pd.Series)
paired_df = paired_df.loc[:, ['dataset_1', 'dataset_2','co_occurence']]
# Only co-occurence > 1
paired_df = paired_df.loc[paired_df.co_occurence > 1]

In [15]:
paired_df.head(2)

,dataset_1,dataset_2,co_occurence
1060,cadastre,pci-vecteur-plan-cadastral-informatise,1682
8450,base-officielle-des-codes-postaux,correspondance-entre-les-codes-postaux-et-code...,413


In [16]:
# Let's build a dict : dataset_A -> (dataset_E : 200), (dataset_B : 130), ...

paired_df_grouped = paired_df.groupby('dataset_1')['dataset_2', 'co_occurence'].agg(lambda x: list(x))
paired_df_grouped.reset_index(inplace=True)
paired_df_grouped['all_co_occurence'] = paired_df_grouped.apply(lambda x: tuple(zip(x.dataset_2, x.co_occurence)), axis=1)
paired_df_grouped.head(2)

,dataset_1,dataset_2,co_occurence,all_co_occurence
0,15-succes-du-programme-national-de-renovation-...,[base-sirene-des-entreprises-et-de-leurs-etabl...,"[4, 4, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2]",((base-sirene-des-entreprises-et-de-leurs-etab...
1,15010-flux-dimmigration-en-provenance-des-pays...,[2013-tableau-ndeg-1-les-etrangers-selon-leur-...,[2],((2013-tableau-ndeg-1-les-etrangers-selon-leur...


### Some datasets are linked with tons of others, let's select only the one that distinguish themselves from others.


To do this will proceed in 2 steps : 

- Select only top10 of co_occurrence : since we want to compute some recommendation on datagouv, we don't need more than 10 linked datasets (less would be nice).

- Apply rule of thumb to select only the best ones (see method below)

In [17]:
def rule_of_thumb(mylist): 
    '''Compute rule of thumb on a list
    eg: [100, 80, 20, 14, 12, 11, 1]  => [100, 80] (return the index where to cut)
    '''
    if len(mylist) > 1:
        # list of drops
        difflist = [mylist[el] - mylist[1:][el] for el in range(0, len(mylist) - 1)]
        max_drop = max(difflist)

        # index of maximum(s)
        max_drop_index = max([i for i, j in enumerate(difflist) if j == max(difflist)])

        return max_drop_index + 1

    return mylist[0]

mylist = [100, 80, 20, 14, 12, 11, 1]
mylist[:rule_of_thumb(mylist)]

[100, 80]

In [18]:
# Sort by co_occurence (max co_occurences in first row of df)
paired_df_grouped.sort_values('co_occurence', ascending=False, inplace=True)
paired_df_grouped.loc[:, ['dataset_1', 'all_co_occurence']].head(20)

# Keep only top10 co-occurence
paired_df_grouped['top10'] = paired_df_grouped.co_occurence.apply(lambda x: x[:10])


In [19]:
paired_df_grouped.head(2)

,dataset_1,dataset_2,co_occurence,all_co_occurence,top10
1100,cadastre,"[pci-vecteur-plan-cadastral-informatise, cadas...","[1682, 145, 137, 100, 94, 54, 49, 48, 42, 39, ...","((pci-vecteur-plan-cadastral-informatise, 1682...","[1682, 145, 137, 100, 94, 54, 49, 48, 42, 39]"
921,base-officielle-des-codes-postaux,[correspondance-entre-les-codes-postaux-et-cod...,"[413, 317, 276, 203, 185, 163, 150, 87, 79, 76...",((correspondance-entre-les-codes-postaux-et-co...,"[413, 317, 276, 203, 185, 163, 150, 87, 79, 76]"


In [20]:
# Apply rule of thumbs to top10 
paired_df_grouped['best_co_occurence'] = paired_df_grouped.apply(lambda x: x['all_co_occurence'][:rule_of_thumb(x['top10'])], axis=1)
del paired_df_grouped['top10']
paired_df_grouped.head(10)


,dataset_1,dataset_2,co_occurence,all_co_occurence,best_co_occurence
1100,cadastre,"[pci-vecteur-plan-cadastral-informatise, cadas...","[1682, 145, 137, 100, 94, 54, 49, 48, 42, 39, ...","((pci-vecteur-plan-cadastral-informatise, 1682...","((pci-vecteur-plan-cadastral-informatise, 1682),)"
921,base-officielle-des-codes-postaux,[correspondance-entre-les-codes-postaux-et-cod...,"[413, 317, 276, 203, 185, 163, 150, 87, 79, 76...",((correspondance-entre-les-codes-postaux-et-co...,((correspondance-entre-les-codes-postaux-et-co...
935,base-sirene-des-entreprises-et-de-leurs-etabli...,[base-sirene-des-entreprises-et-de-leurs-etabl...,"[400, 296, 295, 276, 192, 190, 183, 172, 152, ...",((base-sirene-des-entreprises-et-de-leurs-etab...,((base-sirene-des-entreprises-et-de-leurs-etab...
769,associations,"[repertoire-national-des-associations-rna, ass...","[368, 47, 35, 30, 16, 9, 9, 7, 7, 6, 6, 5, 5, ...","((repertoire-national-des-associations-rna, 36...","((repertoire-national-des-associations-rna, 36..."
3026,immeubles-proteges-au-titre-des-monuments-hist...,[monuments-historiques-liste-des-immeubles-pro...,"[283, 25, 10, 8, 8, 7, 7, 6, 6, 5, 5, 5, 5, 5,...",((monuments-historiques-liste-des-immeubles-pr...,((monuments-historiques-liste-des-immeubles-pr...
1665,contours-des-departements-francais-issus-d-ope...,[decoupage-administratif-communal-francais-iss...,"[261, 173, 51, 48, 36, 35, 34, 34, 29, 26, 21,...",((decoupage-administratif-communal-francais-is...,((decoupage-administratif-communal-francais-is...
1870,decoupage-administratif-communal-francais-issu...,"[geofla-limites-des-communes-en-france, limite...","[236, 210, 121, 107, 89, 70, 69, 62, 60, 53, 5...","((geofla-limites-des-communes-en-france, 236),...","((geofla-limites-des-communes-en-france, 236),..."
1701,correspondance-code-insee-code-postal,[correspondance-entre-les-codes-postaux-et-cod...,"[212, 17, 11, 11, 11, 10, 9, 7, 6, 4, 3, 2, 2,...",((correspondance-entre-les-codes-postaux-et-co...,((correspondance-entre-les-codes-postaux-et-co...
2741,faits-constates-mensuels-par-departements,[faits-constates-par-index-par-departement-et-...,"[210, 33, 30, 25, 25, 22, 21, 19, 19, 18, 17, ...",((faits-constates-par-index-par-departement-et...,((faits-constates-par-index-par-departement-et...
1670,contours-des-regions-francaises-sur-openstreetmap,[decoupage-administratif-communal-francais-iss...,"[196, 76, 63, 59, 40, 38, 26, 22, 20, 19, 16, ...",((decoupage-administratif-communal-francais-is...,((decoupage-administratif-communal-francais-is...


25% of datasets are linked with only 1 other. That's not much but that's enough for now.

We could definitely find a lighter rule if we want to recommend at least 3 datasets.

In [21]:
pd.set_option('display.max_colwidth', -1)
paired_df_grouped[['dataset_1', 'best_co_occurence']]

,dataset_1,best_co_occurence
1100,cadastre,"((pci-vecteur-plan-cadastral-informatise, 1682),)"
921,base-officielle-des-codes-postaux,"((correspondance-entre-les-codes-postaux-et-codes-insee-des-communes-francaises, 413),)"
935,base-sirene-des-entreprises-et-de-leurs-etablissements-siren-siret,"((base-sirene-des-entreprises-et-de-leurs-etablissements-siren-siret, 400),)"
769,associations,"((repertoire-national-des-associations-rna, 368),)"
3026,immeubles-proteges-au-titre-des-monuments-historiques,"((monuments-historiques-liste-des-immeubles-proteges-au-titre-des-monuments-historiques, 283),)"
1665,contours-des-departements-francais-issus-d-openstreetmap,"((decoupage-administratif-communal-francais-issu-d-openstreetmap, 261), (contours-des-regions-francaises-sur-openstreetmap, 173))"
1870,decoupage-administratif-communal-francais-issu-d-openstreetmap,"((geofla-limites-des-communes-en-france, 236), (limites-administratives-francaises-issues-d-openstreetmap, 210))"
1701,correspondance-code-insee-code-postal,"((correspondance-entre-les-codes-postaux-et-codes-insee-des-communes-francaises, 212),)"
2741,faits-constates-mensuels-par-departements,"((faits-constates-par-index-par-departement-et-par-annee, 210),)"
1670,contours-des-regions-francaises-sur-openstreetmap,"((decoupage-administratif-communal-francais-issu-d-openstreetmap, 196),)"


In [22]:
pd.set_option('display.max_colwidth', 100)

# Export

In [23]:
# Convert to dict
datasets_co_occurence = paired_df_grouped[['dataset_1', 'best_co_occurence']].set_index('dataset_1').to_dict()['best_co_occurence']